# Structured Output Tutorial: Book Review Agent

## Overview

Learn how to create AutoGen agents that return **structured, predictable output** instead of free-form text. This makes your AI responses perfect for:

- Database integration
- API responses  
- Automated workflows
- Data validation

**Example**: We'll build a Book Review Agent that analyzes any book description and returns structured ratings and categories.

## Step 1: Define Your Structure

First, create a Pydantic model that defines exactly what data you want back:

In [1]:
from typing import Literal, List
from pydantic import BaseModel

class BookReview(BaseModel):
    title: str
    genre: Literal["fiction", "non-fiction", "mystery", "romance", "sci-fi", "biography", "self-help"]
    rating: Literal[1, 2, 3, 4, 5]  # 1-5 stars
    target_audience: Literal["children", "young-adult", "adult", "academic"]
    themes: List[str]  # Main themes/topics
    recommendation: str  # One sentence recommendation

print("✅ BookReview model defined!")

✅ BookReview model defined!


## Step 2: Create Your Agent

Create an agent that will return your structured format:

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create model and agent (no await needed)
# These just set up objects in memory - no network calls yet!
model_client = OpenAIChatCompletionClient(model="gpt-4o")

book_agent = AssistantAgent(
    name="book_reviewer",
    model_client=model_client,
    system_message="You are a book critic. Analyze books and provide structured reviews in the exact JSON format specified.",
    output_content_type=BookReview,  # This is the magic line!
)

print("🤖 Book Review Agent created!")

🤖 Book Review Agent created!


## Step 3: Test It Out

Send a book description and get structured data back:

In [3]:
from autogen_agentchat.messages import StructuredMessage
from autogen_agentchat.ui import Console

# Test with a book description
book_description = """
"The Martian" by Andy Weir is about an astronaut stranded on Mars who must 
use science and engineering to survive until rescue. It's full of problem-solving, 
humor, and technical details about space survival.
"""

# Get structured response
result = await Console(book_agent.run_stream(task=book_description))

# Validate it's structured (the Console output above shows this worked!)
assert isinstance(result.messages[-1], StructuredMessage)
assert isinstance(result.messages[-1].content, BookReview)

# Extract the structured data
review = result.messages[-1].content

---------- TextMessage (user) ----------

"The Martian" by Andy Weir is about an astronaut stranded on Mars who must 
use science and engineering to survive until rescue. It's full of problem-solving, 
humor, and technical details about space survival.

---------- StructuredMessage[BookReview] (book_reviewer) ----------
{"title":"The Martian","genre":"sci-fi","rating":5,"target_audience":"adult","themes":["survival","isolation","ingenuity","perseverance","humanity"],"recommendation":"A must-read for fans of hard science fiction and adventure stories with a focus on resourcefulness and clever problem-solving. Highly recommended for readers interested in space and science."}


**Understanding the Console output**

Notice how the `Console` [api docs](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.ui.html#autogen_agentchat.ui.Console) displayed the conversation flow above:

1. 📝 TextMessage (user) - Your input book description
2. 🤖 StructuredMessage[BookReview] (book_reviewer) - Agent's structured response

The Console is AutoGen's built-in UI that shows you exactly what's happening
in the conversation. It's perfect for debugging and understanding how your
agents are communicating!


## Step 4: Use the Structured Data

Now you can easily work with the structured output programmatically:

In [4]:
# Easy to filter and categorize
if review.rating >= 4:
    print("✅ Highly recommended book!")
elif review.rating >= 3:
    print("👍 Worth reading")
else:
    print("❌ Skip this one")

# Easy to save to database
book_data = {
    "title": review.title,
    "genre": review.genre,
    "rating": review.rating,
    "themes": review.themes
}

print(f"\n💾 Ready for database: {book_data}")

# Convert to JSON for APIs
json_output = review.model_dump_json()
print(f"\n📡 JSON for API: {json_output}")

✅ Highly recommended book!

💾 Ready for database: {'title': 'The Martian', 'genre': 'sci-fi', 'rating': 5, 'themes': ['survival', 'isolation', 'ingenuity', 'perseverance', 'humanity']}

📡 JSON for API: {"title":"The Martian","genre":"sci-fi","rating":5,"target_audience":"adult","themes":["survival","isolation","ingenuity","perseverance","humanity"],"recommendation":"A must-read for fans of hard science fiction and adventure stories with a focus on resourcefulness and clever problem-solving. Highly recommended for readers interested in space and science."}


## Cleanup

Only the model client cleanup needs await (closes network connection):

In [5]:
await model_client.close()
print("✅ Done!")

✅ Done!


## Key Points

1. **Define structure first**: Create a Pydantic model with the fields you want
2. **Use `output_content_type`**: This tells the agent to return structured data
3. **Validate the response**: Check it's a `StructuredMessage` with your model type
4. **Work programmatically**: Filter, save, convert to JSON easily

**Try it yourself**: 

- Modify the `BookReview` model or create your own for movies, products, or restaurants!
